<a href="https://colab.research.google.com/github/mpmsiva/pysaprk-jupyter-note-book/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,498 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:14 http://archi

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
df=spark.read.csv("sample_data/california_housing_test.csv")


+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|        _c0|      _c1|               _c2|        _c3|           _c4|        _c5|       _c6|          _c7|               _c8|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population|households|median_income|median_house_value|
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000|606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000|277.000000|     3.599000|     176500.000000|
|-117.810000|33.780000|         27.000000|3589.000000|    507.000000|1484.000000|495.000000|     5.793400|     270500.000000|
|-118.360000|33.820000|         28.000000|  67.000000|     15.000000|  49.000000| 11.000000|     6.135900|     330000.

In [ ]:
print(df.count())
len(df.columns)

3001


9

In [ ]:
def spark_shape(self):
  return(self.count(),len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [ ]:
df.shape()

(3001, 9)

In [ ]:
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates=spark.sparkContext.broadcast(states)
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]
columns = ["firstname","lastname","country","state"]
df= spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+



In [ ]:
def state_converter(code):
  return broadcastStates.value[code]

result= df.rdd.map(lambda x: (x[0],x[1],x[2],x[3],state_converter(x[3]))).toDF(columns)
result.show()






+---------+--------+-------+-----+----------+
|firstname|lastname|country|state|        _5|
+---------+--------+-------+-----+----------+
|    James|   Smith|    USA|   CA|California|
|  Michael|    Rose|    USA|   NY|  New York|
|   Robert|Williams|    USA|   CA|California|
|    Maria|   Jones|    USA|   FL|   Florida|
+---------+--------+-------+-----+----------+



In [ ]:

dataDictionary = [
        ('James',{'hair':'black','eye':'brown'}),
        ('Michael',{'hair':'brown','eye':None}),
        ('Robert',{'hair':'red','eye':'black'}),
        ('Washington',{'hair':'red','eye':'grey'}),
        ('Jefferson',{'hair':'red','eye':''})
        ]

In [ ]:
df= spark.createDataFrame(data=dataDictionary, schema=["name","properties"])
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> NULL, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> red}   |
|Jefferson |{eye -> , hair -> red}       |
+----------+-----------------------------+



In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,MapType
schema= StructType([
    StructField('name',StringType(),True),
    StructField('properties',MapType(StringType(),StringType(),True))])
df2=spark.createDataFrame(data=dataDictionary,schema=schema)
df2.show(truncate=False)

df2.printSchema()


+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> NULL, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> red}   |
|Jefferson |{eye -> , hair -> red}       |
+----------+-----------------------------+

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [ ]:
df.rdd.map(lambda x: (x.name,x.properties["hair"],x.properties["eye"])).toDF(["name","hair","eye"]).show()


+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| NULL|
|    Robert|  red|black|
|Washington|  red| grey|
| Jefferson|  red|     |
+----------+-----+-----+



In [ ]:
accum=spark.sparkContext.accumulator(0)
rdd= spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

15


In [ ]:
def countfun(x):
  global accum
  accum+=x
rdd.foreach(countfun)
print(accum.value)

35


In [ ]:
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

# Column names
columns = ["firstname","lastname","country","state"]

# Create DataFrame
df = spark.createDataFrame(data = data, schema = columns)
df.show(truncate=False)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+



In [ ]:
df.select(*columns).show()

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+



In [ ]:
schema = StructType([
    StructField('name', StructType([
         StructField('firstname', StringType(), True),
         StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
         ])),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
     ])


data = [(("James",None,"Smith"),"OH","M"),
        (("Anna","Rose",""),"NY","F"),
        (("Julia","","Williams"),"OH","F"),
        (("Maria","Anne","Jones"),"NY","M"),
        (("Jen","Mary","Brown"),"NY","M"),
        (("Mike","Mary","Williams"),"OH","M")
        ]

df2 = spark.createDataFrame(data = data, schema = schema)
df2.printSchema()
df2.show(truncate=False)






root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+-----+------+
|name                  |state|gender|
+----------------------+-----+------+
|{James, NULL, Smith}  |OH   |M     |
|{Anna, Rose, }        |NY   |F     |
|{Julia, , Williams}   |OH   |F     |
|{Maria, Anne, Jones}  |NY   |M     |
|{Jen, Mary, Brown}    |NY   |M     |
|{Mike, Mary, Williams}|OH   |M     |
+----------------------+-----+------+



In [ ]:
df2.select('name.*',"state","gender").show(truncate=False)

+---------+----------+--------+-----+------+
|firstname|middlename|lastname|state|gender|
+---------+----------+--------+-----+------+
|James    |NULL      |Smith   |OH   |M     |
|Anna     |Rose      |        |NY   |F     |
|Julia    |          |Williams|OH   |F     |
|Maria    |Anne      |Jones   |NY   |M     |
|Jen      |Mary      |Brown   |NY   |M     |
|Mike     |Mary      |Williams|OH   |M     |
+---------+----------+--------+-----+------+



In [ ]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]

In [ ]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("siva").getOrCreate()

In [ ]:
df= spark.createDataFrame(data=data,schema=columns)
df.show(truncate=False)

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|dob       |gender|salary|
+---------+----------+--------+----------+------+------+
|James    |          |Smith   |1991-04-01|M     |3000  |
|Michael  |Rose      |        |2000-05-19|M     |4000  |
|Robert   |          |Williams|1978-09-05|M     |4000  |
|Maria    |Anne      |Jones   |1967-12-01|F     |4000  |
|Jen      |Mary      |Brown   |1980-02-17|F     |-1    |
+---------+----------+--------+----------+------+------+



In [ ]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
from pyspark.sql.functions import col
df.withColumn("salary",col("salary").cast("Integer")).show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [ ]:
df.withColumn("salary",col("salary")*100).show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|300000|
|  Michael|      Rose|        |2000-05-19|     M|400000|
|   Robert|          |Williams|1978-09-05|     M|400000|
|    Maria|      Anne|   Jones|1967-12-01|     F|400000|
|      Jen|      Mary|   Brown|1980-02-17|     F|  -100|
+---------+----------+--------+----------+------+------+



In [ ]:
df.withColumn("new_salary",col("salary")*12).show()

+---------+----------+--------+----------+------+------+----------+
|firstname|middlename|lastname|       dob|gender|salary|new_salary|
+---------+----------+--------+----------+------+------+----------+
|    James|          |   Smith|1991-04-01|     M|  3000|     36000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|     48000|
|   Robert|          |Williams|1978-09-05|     M|  4000|     48000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|     48000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|       -12|
+---------+----------+--------+----------+------+------+----------+



In [ ]:
from pyspark.sql.functions import lit
df.withColumn("country",lit("india")).show()
df.withColumn("country",lit("india"))\
  .withColumn("state",lit("TN"))\
  .show()

+---------+----------+--------+----------+------+------+-------+
|firstname|middlename|lastname|       dob|gender|salary|country|
+---------+----------+--------+----------+------+------+-------+
|    James|          |   Smith|1991-04-01|     M|  3000|  india|
|  Michael|      Rose|        |2000-05-19|     M|  4000|  india|
|   Robert|          |Williams|1978-09-05|     M|  4000|  india|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|  india|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|  india|
+---------+----------+--------+----------+------+------+-------+

+---------+----------+--------+----------+------+------+-------+-----+
|firstname|middlename|lastname|       dob|gender|salary|country|state|
+---------+----------+--------+----------+------+------+-------+-----+
|    James|          |   Smith|1991-04-01|     M|  3000|  india|   TN|
|  Michael|      Rose|        |2000-05-19|     M|  4000|  india|   TN|
|   Robert|          |Williams|1978-09-05|     M|  4000|  i

In [ ]:
df.withColumnRenamed("gender","sex").show()

+---------+----------+--------+----------+---+------+
|firstname|middlename|lastname|       dob|sex|salary|
+---------+----------+--------+----------+---+------+
|    James|          |   Smith|1991-04-01|  M|  3000|
|  Michael|      Rose|        |2000-05-19|  M|  4000|
|   Robert|          |Williams|1978-09-05|  M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|  F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|  F|    -1|
+---------+----------+--------+----------+---+------+



In [ ]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

schema= StructType([
    StructField('fname',StructType([
        StructField('fname',StringType(),True),
        StructField('mname',StringType(),True),
        StructField('lname',StringType(),True),
        ])),
    StructField('dob',StringType(),True),
    StructField('gender',StringType(),True),
    StructField('salary',StringType(),True),
    ])

dataDF = [(('James','','Smith'),'1991-04-01','M',3000),
  (('Michael','Rose',''),'2000-05-19','M',4000),
  (('Robert','','Williams'),'1978-09-05','M',4000),
  (('Maria','Anne','Jones'),'1967-12-01','F',4000),
  (('Jen','Mary','Brown'),'1980-02-17','F',-1)
]

In [ ]:
df= spark.createDataFrame(data=dataDF,schema=schema).show()

+--------------------+----------+------+------+
|               fname|       dob|gender|salary|
+--------------------+----------+------+------+
|    {James, , Smith}|1991-04-01|     M|  3000|
|   {Michael, Rose, }|2000-05-19|     M|  4000|
|{Robert, , Williams}|1978-09-05|     M|  4000|
|{Maria, Anne, Jones}|1967-12-01|     F|  4000|
|  {Jen, Mary, Brown}|1980-02-17|     F|    -1|
+--------------------+----------+------+------+



In [ ]:
import os
path=os.getcwd()
print(path)

/content


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark = SparkSession.builder.appName("siva").getOrCreate()

In [ ]:
data = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]

# Create DataFrame
column= ["employee_name", "department", "salary"]

df = spark.createDataFrame(data=data, schema=column)
df.show()x

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [ ]:
disdf=df.distinct()
print(disdf.count())

disdf.show()



9
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|      Michael|     Sales|  4600|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Scott|   Finance|  3300|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [ ]:
dupdf=df.dropDuplicates()
print(dupdf.count())

dupdf.show()

9
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|      Michael|     Sales|  4600|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Scott|   Finance|  3300|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|         Saif|     Sales|  4100|
+-------------+----------+------+

